### TensorFlow & Keras based NN Training

In [1]:
import numpy
import pandas
import glob
import logging
from collections import OrderedDict as od
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
data=od()
for filename in glob.glob('../Data/Features/*'):
    key=filename.split('/')[-1].split('_')[0]
    print(key)
    data[key]=pandas.read_csv(filename).reset_index().dropna()

201711092033
201711091951
201711182141
201711161638


In [4]:
def splitX_Y(df):
    X=df[[u'lastAccelerometerValues[0]_mean_D',
       u'lastAccelerometerValues[1]_mean_D', u'lastAccelerometerValues[2]_mean_D',
       u'lastAccelerometerValues[0]_var_D', u'lastAccelerometerValues[1]_var_D',
       u'lastAccelerometerValues[2]_var_D', u'lastAccelerometerValues[0]_skew_D',
       u'lastAccelerometerValues[1]_skew_D', u'lastAccelerometerValues[2]_skew_D',
       u'lastAccelerometerValues[0]_kurtosis_D',
       u'lastAccelerometerValues[1]_kurtosis_D',
       u'lastAccelerometerValues[2]_kurtosis_D', u'lastGyroscopeValues[0]_mean_D',
       u'lastGyroscopeValues[1]_mean_D', u'lastGyroscopeValues[2]_mean_D',
       u'lastGyroscopeValues[0]_var_D', u'lastGyroscopeValues[1]_var_D',
       u'lastGyroscopeValues[2]_var_D', u'lastGyroscopeValues[0]_skew_D',
       u'lastGyroscopeValues[1]_skew_D', u'lastGyroscopeValues[2]_skew_D',
       u'lastGyroscopeValues[0]_kurtosis_D', u'lastGyroscopeValues[1]_kurtosis_D',
       u'lastGyroscopeValues[2]_kurtosis_D', u'lastMagnetometerValues[0]_mean_D',
       u'lastMagnetometerValues[1]_mean_D', u'lastMagnetometerValues[2]_mean_D',
       u'lastMagnetometerValues[0]_var_D', u'lastMagnetometerValues[1]_var_D',
       u'lastMagnetometerValues[2]_var_D', u'lastMagnetometerValues[0]_skew_D',
       u'lastMagnetometerValues[1]_skew_D', u'lastMagnetometerValues[2]_skew_D',
       u'lastMagnetometerValues[0]_kurtosis_D',
       u'lastMagnetometerValues[1]_kurtosis_D',
       u'lastMagnetometerValues[2]_kurtosis_D',
         u'lastAccelerometerValues[0]_mean_U',
       u'lastAccelerometerValues[1]_mean_U', u'lastAccelerometerValues[2]_mean_U',
       u'lastAccelerometerValues[0]_var_U', u'lastAccelerometerValues[1]_var_U',
       u'lastAccelerometerValues[2]_var_U', u'lastAccelerometerValues[0]_skew_U',
       u'lastAccelerometerValues[1]_skew_U', u'lastAccelerometerValues[2]_skew_U',
       u'lastAccelerometerValues[0]_kurtosis_U',
       u'lastAccelerometerValues[1]_kurtosis_U',
       u'lastAccelerometerValues[2]_kurtosis_U', u'lastGyroscopeValues[0]_mean_U',
       u'lastGyroscopeValues[1]_mean_U', u'lastGyroscopeValues[2]_mean_U',
       u'lastGyroscopeValues[0]_var_U', u'lastGyroscopeValues[1]_var_U',
       u'lastGyroscopeValues[2]_var_U', u'lastGyroscopeValues[0]_skew_U',
       u'lastGyroscopeValues[1]_skew_U', u'lastGyroscopeValues[2]_skew_U',
       u'lastGyroscopeValues[0]_kurtosis_U', u'lastGyroscopeValues[1]_kurtosis_U',
       u'lastGyroscopeValues[2]_kurtosis_U', u'lastMagnetometerValues[0]_mean_U',
       u'lastMagnetometerValues[1]_mean_U', u'lastMagnetometerValues[2]_mean_U',
       u'lastMagnetometerValues[0]_var_U', u'lastMagnetometerValues[1]_var_U',
       u'lastMagnetometerValues[2]_var_U', u'lastMagnetometerValues[0]_skew_U',
       u'lastMagnetometerValues[1]_skew_U', u'lastMagnetometerValues[2]_skew_U',
       u'lastMagnetometerValues[0]_kurtosis_U',
       u'lastMagnetometerValues[1]_kurtosis_U',
       u'lastMagnetometerValues[2]_kurtosis_U']]
    Y=df[['target']]
    return X, Y

In [5]:
X=pandas.DataFrame()
Y=pandas.DataFrame()
for key in data.keys():
    x,y=splitX_Y(data[key])
    X=X.append(x)
    Y=Y.append(y)
X=X.reset_index().drop(['index'],axis=1)
Y=Y.reset_index().drop(['index'],axis=1)

In [6]:
Y0=Y.applymap(lambda x:0 if x<72 else 1)

In [7]:
Y1=Y.applymap(lambda x: x//36)

In [8]:
Y2=Y.applymap(lambda x: x//18)

In [9]:
Y3=Y.applymap(lambda x: x//9)

In [10]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y0)
encoded_Y0 = encoder.transform(Y0)

/usr/local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y1)
encoded_Y1 = encoder.transform(Y1)

In [12]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y2)
encoded_Y2 = encoder.transform(Y2)

In [13]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y3)
encoded_Y3 = encoder.transform(Y3)

In [14]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y0 = np_utils.to_categorical(encoded_Y0)
dummy_y1 = np_utils.to_categorical(encoded_Y1)
dummy_y2 = np_utils.to_categorical(encoded_Y2)
dummy_y3 = np_utils.to_categorical(encoded_Y3)

### Model 0
36 inputs -> [40 hidden nodes] -> 2 outputs

In [16]:
# define baseline model
def model0():
    # create model
    model = Sequential()
    model.add(Dense(80, input_dim=72, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## Model 1
36 inputs -> [40 hidden nodes] -> 4 outputs

In [17]:
# define baseline model
def model1():
    # create model
    model = Sequential()
    model.add(Dense(80, input_dim=72, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## Model 2
36 inputs -> [40 hidden nodes] -> 8 outputs

In [18]:
# define baseline model
def model2():
    # create model
    model = Sequential()
    model.add(Dense(80, input_dim=72, activation='relu'))
    model.add(Dense(8, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## Model 3
36 inputs -> [40 hidden nodes] -> 16 outputs

In [19]:
# define baseline model
def model3():
    # create model
    model = Sequential()
    model.add(Dense(80, input_dim=72, activation='relu'))
    model.add(Dense(16, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [20]:
estimator0 = KerasClassifier(build_fn=model0, epochs=200, batch_size=5, verbose=0)
estimator1 = KerasClassifier(build_fn=model1, epochs=200, batch_size=5, verbose=0)
estimator2 = KerasClassifier(build_fn=model2, epochs=200, batch_size=5, verbose=0)
estimator3 = KerasClassifier(build_fn=model3, epochs=200, batch_size=5, verbose=0)

In [21]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [ ]:
%%time
results0 = cross_val_score(estimator0, X.values, dummy_y0, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
%%time
results1 = cross_val_score(estimator1, X.values, dummy_y1, cv=k2fold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
%%time
results2 = cross_val_score(estimator2, X.values, dummy_y2, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
%%time
results3 = cross_val_score(estimator3, X.values, dummy_y3, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
history = estimator0.model.fit(X, Y0, validation_split=0.33, epochs=150, batch_size=10, verbose=0)

In [ ]:
estimator2.model.fit(X.values,Y2.values)

In [ ]:
import pydotplus

# import theano.d3viz as d3v
d3v.d3viz(estimator.get_output(), 'test.html')

In [ ]:
hist0=estimator0.fit(X,Y0)

In [ ]:
# Compute the correlation matrix
corr = x.corr()

# Generate a mask for the upper triangle
mask = numpy.zeros_like(corr, dtype=numpy.bool)
mask[numpy.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.show()

In [ ]:
plt.show()